# [Module 4.3] Personalize 캠페인과 실시간 상호 작용 하기

이 노트북은 사용자의 실시간 행동에 반응하는 기능을 추가하는 과정을 안내합니다. 영화를 탐색하는 동안 사용자의 의도가 변경되면, 해당 동작에 따라 수정된 추천 영화 목록들이 표시됩니다.
이 노트북은 다음과 같은 작업을 합니다.
- 이벤트 추적기 생성
- 이전 노트북에서 실행한 처음 추천 결과 보기
- 새로운 영화를 클릭했다고 가정하고, 이벤트 트랙커 업데이트 후에 두 번째 추천 결과 보기
- 다른 새로운 영화를 클릭했다고 가정하고, 이벤트 트랙커 업데이트 후에 세 번째 추천 결과 보기

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid
from random import randint

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r


생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [4]:
# suffix = str(np.random.uniform())[4:9]

In [5]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## 이벤트 추적기 생성

추천 시스템이 실시간 이벤트에 응답하기 전에 이벤트 추적기(Event Tracker)가 필요합니다. 아래 코드 셀에서 이벤트 추적기 하나를 생성하고 이 실습에서 계속 사용하겠습니다. 이벤트 추적기 이름을 `MovieClickTracker`로 부여하였지만, 여러분들이 자유롭게 지정하실 수 있습니다.<br>
위의 추적 ID가 표시되며, 이 ID는 변수에 할당되었으므로 추가 조치가 필요하지 않습니다. 아래 코드 셀을 실행하면 나중에 추천 영화 목록을 출력할 수 있도록 합니다.

In [5]:
dataset_group_arn

'arn:aws:personalize:ap-northeast-2:057716757052:dataset-group/RetailDemo-dataset-group-18036'

In [6]:
import time


response = personalize.create_event_tracker(
    name='MovieClickTracker',
    datasetGroupArn=dataset_group_arn
)

# wait tracker creation
time.sleep(30)

print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId'] 

arn:aws:personalize:ap-northeast-2:057716757052:event-tracker/44551a3e
045e0d46-327e-43aa-8d0a-1b6407502759


In [8]:
event_tracker_arn = response['eventTrackerArn']

## 처음 추천 결과 확인

이전 노트북에서 아래 유저에 대해서 (예: 4375)  추천 리스트를 확인 합니다. <br>

In [7]:
user_id = 1
print("user_id: ", user_id)

user_id:  1


In [8]:
from src.p_utils import get_item_list_details

items_df = pd.read_csv(items_filename)


get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = MostViewed_RecommenderARN,
    userId = str(user_id),
    numResults = 10
)

print("Recommended items")

items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
get_item_list_details(items_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름

Recommended items


,ITEM_ID,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,7bc976b5-c78c-42aa-a4b2-dd734ce1047f,Fancy Wood Bowl,housewares,bowls,This fancy wood bowl is a must-have,67.99
1,898e919e-4758-41bf-bc8a-f96bcf7e375b,Magnificent Ceramic Bowl,housewares,bowls,This magnificent ceramic bowl is a must-have,23.99
2,05f4f79c-e6d1-4c31-bcfe-44d7e77fcb68,Strainer,housewares,kitchen,A must-have for your kitchen,68.99
3,dc8f6e1b-3d49-436e-93ef-821fe08ffa4e,Coffee Maker,housewares,kitchen,Unrivaled for every kitchen,53.99
4,ab228b40-f692-4662-9986-6d8184dda20b,Beauty Balm,beauty,bathing,This product is bottled heaven,51.99
5,6edbdde6-0791-4a53-ae92-b8aaba88753d,Chef Knife,housewares,kitchen,Incomparable for every kitchen,72.99
6,a3ad25e8-c9c3-459a-a853-c26a0ad837eb,Gainsboro Backpack,accessories,backpack,This gainsboro backpack is incomparable for th...,133.99
7,912fb371-de19-4753-b43a-87bf4b18bac2,Chef Knife,housewares,kitchen,A must-have for your kitchen,51.99
8,5a9e66ed-32c5-461e-a4b0-a56948c3235b,Black Leather Bag,accessories,bag,An all-around convenient bag for everyday use,96.99
9,63074efc-388e-4505-b984-5b25a4441299,Sublime Soap,beauty,bathing,Enjoy the fragrance of this sublime soap,22.99


## 사용자 행동 시뮬레이션

아래 코드 셀은 특정 item과 상호 작용하는 사용자를 시뮬레이트하는 코드 샘플을 제공하며, 시작할 때와 다른 추천 목록을 얻습니다.

아래 ```send_movie_click(USER_ID, ITEM_ID)``` 는 다음과 같은 작업을 하여 실시간 이벤트를 반영 합니다.

- 유저에 대한 세션이 없으면 SessionID를 생성하고, 있으면 이 유저의 SessionID를 불러온다
- event를 정의하고 event에 item_id를 할당한다
- event를 json 형태로 바꾼다
- 위에서 생성한 event tranker의 put_event에 위 json파일을 인자로 넘겨서 호출한다.

#### Help 함수

In [15]:
session_dict = {}

def send_click(EVENT_TYPE, USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
   
    # value=randint(0,5)
    
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    #"eventValue": value
    }
    event_json = json.dumps(event)
    
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID, # 이벤트트래커에서 생성한 아이디
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': EVENT_TYPE,
        'properties': event_json
        }]
)
    
# Help 함수

# def rec_item_list(item_list, clicked_title, clicked_genre):
#     recommendation_title_list = []
#     recommendation_genre_list = []

#     for item in item_list:
#         title = get_movie_title(item['itemId'])
#         genre = get_movie_genre(item['itemId'])
#         recommendation_title_list.append(title)
#         recommendation_genre_list.append(genre)    

#     title_df = pd.DataFrame(recommendation_title_list, columns=[clicked_title])
#     genre_df = pd.DataFrame(recommendation_genre_list, columns=[clicked_genre])        
#     recommendations_df = title_df.join(genre_df)

#     return recommendations_df                                                    

# def get_new_recommend(user_id,campaign_arn, recommendations_df, title, genre):    
#     get_recommendations_response = personalize_runtime.get_recommendations(
#         campaignArn = campaign_arn,
#         userId = str(user_id),
#     )

#     item_list = get_recommendations_response['itemList']
#     new_recs_df = rec_item_list(item_list,title,  genre)
#     new_recommendations_df = pd.concat([recommendations_df,new_recs_df ], axis=1)

#     print("Recommendations for user: ", user_id)
#     return new_recommendations_df
    

# 실시간 클릭 전달

In [21]:
%store -r
pd.options.display.max_columns = 10


EVENT_TYPE = 'View'
user_id = 1
item_id = '4952a24b-7c7c-4ee6-a448-720e97927ba8' # Cool Watch

# 이벤트 추적기 put_event 호출

import time
click_nums = 100

for i in range(click_nums):
    send_click(EVENT_TYPE, USER_ID=str(user_id), ITEM_ID=item_id)
    time.sleep(1)
# 새로운 추천 결과


In [6]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = MostViewed_RecommenderARN,
    userId = str(user_id),
    numResults = 10
)

print("Recommended items")

items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
get_item_list_details(items_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름

Recommended items


NameError: name 'get_item_list_details' is not defined

In [27]:
print("test")

test


## 결론

사용자가 상호 작용하는 영화를 변경하여 추천 사항이 변경되었음을 알 수 있습니다. 이 시스템은 사용자가 item 모음과 상호 작용하는 모든 응용 프로그램으로 확장할 수 있습니다. 이러한 도구는 언제든지 사용 가능한 데이터로 가능한 것을 풀다운하여 탐색할 수 있습니다.

In [24]:
%store event_tracker_arn

Stored 'event_tracker_arn' (str)
